<a href="https://colab.research.google.com/github/rageya/food-vision/blob/main/FoodVision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Continue with regular imports
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

# Try to import the going_modular directory, download it from GitHub if it doesn't work
try:
    from going_modular.going_modular import data_setup, engine
    from helper_functions import download_data, set_seeds, plot_loss_curves
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular or helper_functions scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !mv pytorch-deep-learning/helper_functions.py . # get the helper_functions.py script
    !rm -rf pytorch-deep-learning
    from going_modular.going_modular import data_setup, engine
    from helper_functions import download_data, set_seeds, plot_loss_curves

In [ ]:
device  = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
# Download pizza, steak, sushi images from GitHub
data_20_percent_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip",
                                     destination="pizza_steak_sushi_20_percent")

data_20_percent_path

[INFO] data/pizza_steak_sushi_20_percent directory exists, skipping download.


PosixPath('data/pizza_steak_sushi_20_percent')

In [ ]:
train_dir = data_20_percent_path / 'train'
test_dir = data_20_percent_path / 'test'

## Creating an EffNetB2 faeture extractor

Feature extractor : a term for a trasnfer learning model that has its base layers frozen and output or head layers customized to a cetain problem.

In [ ]:
# setup pretrained EffNetB2 weights
effnetb2_weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT # best avaialable weights

# get EffNetB2 transforms
effnetb2_transforms = effnetb2_weights.transforms()

#3 setup pretrained model instance
effnetb2 = torchvision.models.efficientnet_b2(weights=effnetb2_weights).to(device)

# freeze the base layers in the model(will stop all layers from training)
for param in effnetb2.parameters():
    param.requires_grad = False

In [ ]:
effnetb2.classifier

Sequential(
  (0): Dropout(p=0.3, inplace=True)
  (1): Linear(in_features=1408, out_features=1000, bias=True)
)

In [ ]:
# set seeds for reroducibility
set_seeds()
effnetb2.classifier = nn.Sequential(
    nn.Dropout(p=0.3,
               inplace=True),
    nn.Linear(in_features=1408,
              out_features=3,
              bias=True)
)

In [ ]:
from torchinfo import summary

summary(effnetb2,
        col_names=["input_size", "output_size", "num_params", "trainable"],input_size=(1, 3, 224, 224),
         col_width=20,
        row_settings=["var_names"]
        )

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [1, 3, 224, 224]     [1, 3]               --                   Partial
├─Sequential (features)                                      [1, 3, 224, 224]     [1, 1408, 7, 7]      --                   False
│    └─Conv2dNormActivation (0)                              [1, 3, 224, 224]     [1, 32, 112, 112]    --                   False
│    │    └─Conv2d (0)                                       [1, 3, 224, 224]     [1, 32, 112, 112]    (864)                False
│    │    └─BatchNorm2d (1)                                  [1, 32, 112, 112]    [1, 32, 112, 112]    (64)                 False
│    │    └─SiLU (2)                                         [1, 32, 112, 112]    [1, 32, 112, 112]    --                   --
│    └─Sequential (1)                                        [1, 32, 112, 112]    [1, 1

In [ ]:
def create_effnetb2_model(num_classes:int=3,
                          seed:int=42):
    """Creates an EfficientNetB2 feature extractor model and transforms.

    Args:
        num_classes (int, optional): number of classes in the classifier head.
            Defaults to 3.
        seed (int, optional): random seed value. Defaults to 42.

    Returns:
        model (torch.nn.Module): EffNetB2 feature extractor model.
        transforms (torchvision.transforms): EffNetB2 image transforms.
    """
    # Create EffNetB2 pretrained weights, transforms and model
    # Changed: Use torchvision.models instead of torchvision.model
    weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
    transforms = weights.transforms()
    model = torchvision.models.efficientnet_b2(weights=weights)

    # Freeze all layers in base model
    for param in model.parameters():
        param.requires_grad = False

    # Change classifier head with random seed for reproducibility
    torch.manual_seed(seed)
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.3, inplace=True),
        nn.Linear(in_features=1408, out_features=num_classes),
    )

    return model, transforms

 ### Creating Dataloaders for EffNetB2

In [ ]:
from going_modular.going_modular import data_setup
train_dataloader_effnetb2, test_dataloader_effnetb2, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                                                 test_dir=test_dir,
                                                                                                 transform =effnetb2_transforms,
                                                                                                 batch_size= 32)

### Training an Effnetb2 model


In [ ]:
from going_modular.going_modular import engine

# loss fn
loss_fn = nn.CrossEntropyLoss()

# optimizer
optimizer = torch.optim.Adam(params=effnetb2.parameters(),
                              lr=1e-3)

# training function
set_seeds()
effnet_results = engine.train(model=effnetb2,
                              train_dataloader=train_dataloader_effnetb2,
                              test_dataloader = test_dataloader_effnetb2,
                              epochs=10,
                              optimizer=optimizer,
                              loss_fn=loss_fn,
                              device=device)

  0%|          | 0/10 [00:00<?, ?it/s]

TypeError: train_step() missing 1 required positional argument: 'epochs'

### Inspecting EffNetb2 Loss curves..

In [ ]:
plot_loss_curves(effnet_results)

In [ ]:
### Saving EffNetB2 feature extractor
from going_modular.going_modular import utils

utils.save_model(model=effnetb2,
                 target_dir='models',
                 model_name='pretrained_effnetb2_feature_extractor_20%.pth')

### Inspecting the size o f our EffNetB2 feature extractor..

In [ ]:
import os
from pathlib import Path
# model size in bytes and convert to megabytes..
file_path = "models/pretrained_effnetb2_feature_extractor_20%.pth"
pretrained_effnetb2_model_size = os.stat(file_path).st_size / (1024 * 1024)
print(f'Pretrained EffNetB2 model size: {pretrained_effnetb2_model_size:.2f} MB')

### Collecting EffNetB2 feature extractor stats

In [ ]:
# count number of parameters in EffNetB2
effnetb2_total_params = sum(torch.numel(param) for param in effnetb2.parameters())
effnetb2_total_params

In [ ]:
# save to dictionary
effnetb2_stats = {"test_loss" : effnet_results["test_loss"][-1],
                  "test_acc" : effnet_results["test_acc"][-1],
                  "number of parameters" : effnetb2_total_params,
                  "model size in MB" : pretrained_effnetb2_model_size}

effnetb2_stats

## Creating a ViT feature extractor

In [ ]:
# Check out ViT's heads layers

vit = torchvision.models.vit_b_16()
vit.heads


In [ ]:
def create_vit_model(num_classes:int=3,
                     seed:int=42):
    # Create ViT_B_16 pretrained weights, transforms and model
    weights = torchvision.models.ViT_B_16_Weights.DEFAULT
    transforms = weights.transforms()
    model = torchvision.models.vit_b_16(weights=weights)

    # Freeze all layers in model
    for param in model.parameters():
        param.requires_grad = False

    # Change classifier head to suit our needs (this will be trainable)
    torch.manual_seed(seed)
    model.heads = nn.Sequential(nn.Linear(in_features=768,
                                          out_features=num_classes))

    return model, transforms

In [ ]:
vit, vit_transforms = create_vit_model()

In [ ]:
vit_transforms

In [ ]:
from torchinfo import summary

  # Print ViT feature extractor model summary (uncomment for full output)
summary(vit,
        input_size=(1, 3, 224, 224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

### Create dataloaders for vit feature extractor

In [ ]:
from going_modular.going_modular import data_setup

train_dataloader_vit, test_dataloader_vit, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                               test_dir=test_dir,
                                                                               transform=vit_transforms,
                                                                               batch_size=32)

In [ ]:
from going_modular.going_modular import engine

# Setup optimizer
optimizer = torch.optim.Adam(params=vit.parameters(),
                             lr=1e-3)
# Setup loss function
loss_fn = torch.nn.CrossEntropyLoss()

# Train ViT model with seeds set for reproducibility
set_seeds()
vit_results = engine.train(model=vit,
                           train_dataloader=train_dataloader_vit,
                           test_dataloader=test_dataloader_vit,
                           epochs=10,
                           optimizer=optimizer,
                           loss_fn=loss_fn,
                           device=device)

In [ ]:
# plot loss curves
plot_loss_curves(vit_results)

In [ ]:
### aving Vit feature xetractor

from going_modular.going_modular import utils

utils.save_model(model=vit,
                 target_dir='models',
                 model_name='pretrained_vit_feature_extractor_20%.pth')

In [ ]:
from pathlib import Path

# get the model size in megabytes
file_path = Path("models/pretrained_vit_feature_extractor_20%.pth")
pretrain_vit_feature_extractor_model_size = Path(file_path).stat().st_size / (1024 * 1024)
print(f'Pre trained ViT feature extractor model size: {pretrain_vit_feature_extractor_model_size:.2f} MB')

### Collecting Vit feature extractor stats

In [ ]:
# count total number of params
vit_total_params = sum(torch.numel(param) for param in vit.parameters())
vit_total_params

In [ ]:
# create ViT stat dictionary
vit_stats = {"test_loss" : vit_results["test_loss"][-1],
             "test_acc" : vit_results["test_acc"][-1],
             "number of parameters" : vit_total_params,
             "model size in MB" : pretrain_vit_feature_extractor_model_size}

vit_stats

In [ ]:
effnetb2_stats

## Making prediction with out trained model and timing..

In [ ]:
from pathlib import Path

# get all test data paths
test_data_paths  = list(Path(test_dir).glob("*/*.jpg"))
test_data_paths[:10]

### Creatin a function to make accros the test dataset

In [ ]:
import pathlib
from PIL import Image
from timeit import default_timer as timer
from tqdm.auto import tqdm

from typing import List, Dict
def pred_and_store(paths: List[pathlib.Path],
                   model: torch.nn.Module,
                   transform: torchvision.transforms,
                   class_Names: List[str],
                   device: str = 'cuda' if torch.cuda.is_available() else 'cpu'):
  # create an empty list
  pred_list = []

  # loop throught the input paths
  for path in tqdm(paths):

    pred_dict = {}

    # get the sample path and ground truth
    pred_dict['image_path'] = path
    class_Name = path.parent.stem
    pred_dict['class_name'] = class_Name

    # sart the pred timer
    start_timer = timer()

    # open the image
    img = Image.open(path)

    # transform the image to be usabele with a given model
    transformed_image = transform(img).unsqueeze(0).to(device)

    model = model.to(device)
    model.eval()

    # TUrn on inference mode
    with torch.inference_mode():
      pred_logits = model(transformed_image)
      pred_prob = torch.softmax(pred_logits, dim=1)
      pred_label = torch.argmax(pred_prob, dim=1)
      pred_class = class_names[pred_label.cpu()] # hardcode pred class to be on cpu as python varibales live on cpu

      # add pred prob + pred class to empty dict
      pred_dict['pred_prob'] = round(pred_prob.unsqueeze(0).max().cpu().item(), 4)
      pred_dict['pred_class'] = pred_class

      # ending the pred timer
      end_timer = timer()
      pred_dict['time_for_pred'] = round(end_timer-start_timer, 4)

    # see if pred class matches the ground truth class
    pred_dict['correct'] = class_Name == pred_class

    # add pred dict to pred list
    pred_list.append(pred_dict)

  return pred_list

### Making and timing pred with EffNetB2

In [ ]:
# Make predictions across test dataset with EffNetB2
effnetb2_test_pred_dicts = pred_and_store(paths=test_data_paths,
                                          model=effnetb2,
                                          transform=effnetb2_transforms,
                                          class_Names=class_names,
                                          device="cpu") # make predictions on CPU

In [ ]:
# Inspect the first 2 prediction dictionaries
effnetb2_test_pred_dicts[:2]

In [ ]:
# Turn the test_pred_dicts into a DataFrame
import pandas as pd
effnetb2_test_pred_df = pd.DataFrame(effnetb2_test_pred_dicts)
effnetb2_test_pred_df.head()

In [ ]:
# Check number of correct predictions
effnetb2_test_pred_df.correct.value_counts()

In [ ]:
# Find the average time per prediction
effnetb2_average_time_per_pred = round(effnetb2_test_pred_df.time_for_pred.mean(), 4)
print(f"EffNetB2 average time per prediction: {effnetb2_average_time_per_pred} seconds")

In [ ]:
# Add EffNetB2 average prediction time to stats dictionary
effnetb2_stats["time_per_pred_cpu"] = effnetb2_average_time_per_pred
effnetb2_stats

### Making timing and pred with ViT

In [ ]:
# Make list of prediction dictionaries with ViT feature extractor model on test images
vit_test_pred_dicts = pred_and_store(paths=test_data_paths,
                                     model=vit,
                                     transform=vit_transforms,
                                     class_Names=class_names,
                                     device="cpu")

In [ ]:
# Check the first couple of ViT predictions on the test dataset
vit_test_pred_dicts[:2]

In [ ]:
# Turn vit_test_pred_dicts into a DataFrame
import pandas as pd
vit_test_pred_df = pd.DataFrame(vit_test_pred_dicts)
vit_test_pred_df.head()

In [ ]:
# Count the number of correct predictions
vit_test_pred_df.correct.value_counts()

In [ ]:
# Calculate average time per prediction for ViT model
vit_average_time_per_pred = round(vit_test_pred_df.time_for_pred.mean(), 4)
print(f"ViT average time per prediction: {vit_average_time_per_pred} seconds")

In [ ]:
# Add average prediction time for ViT model on CPU
vit_stats["time_per_pred_cpu"] = vit_average_time_per_pred
vit_stats

In [ ]:
# Turn stat dictionaries into DataFrame
df = pd.DataFrame([effnetb2_stats, vit_stats])

# Add column for model names
df["model"] = ["EffNetB2", "ViT"]

# Convert accuracy to percentages
df["test_acc"] = round(df["test_acc"] * 100, 2)

df

In [ ]:
# Compare ViT to EffNetB2 across different characteristics
pd.DataFrame(data=(df.set_index("model").loc["ViT"] / df.set_index("model").loc["EffNetB2"]), # divide ViT statistics by EffNetB2 statistics
             columns=["ViT to EffNetB2 ratios"]).T

## Vit VS EffNetB2

In [ ]:
import os
# 1. Create a plot from model comparison DataFrame
fig, ax = plt.subplots(figsize=(12, 8))
scatter = ax.scatter(data=df,
                     x="time_per_pred_cpu",
                     y="test_acc",
                     c=["blue", "orange"], # what colours to use?
                     s=df["model size in MB"]) # size the dots by the model sizes

ax.set_title("FoodVision Mini Inference Speed vs Performance", fontsize=18)
ax.set_xlabel("Prediction time per image (seconds)", fontsize=14)
ax.set_ylabel("Test accuracy (%)", fontsize=14)
ax.tick_params(axis='both', labelsize=12)
ax.grid(True)

#Annotate with model names
for index, row in df.iterrows():
    ax.annotate(text=row["model"],
                xy=(row["time_per_pred_cpu"]+0.0006, row["test_acc"]+0.03),
                size=12)

#Create a legend based on model sizes
handles, labels = scatter.legend_elements(prop="sizes", alpha=0.5)
model_size_legend = ax.legend(handles,
                              labels,
                              loc="lower right",
                              title="Model size (MB)",
                              fontsize=12)
# Create the 'images' directory if it doesn't exist
os.makedirs("images", exist_ok=True)
plt.savefig("images/09-foodvision-mini-inference-speed-vs-performance.jpg")

plt.show()

## Creating a GRADIO demo

In [ ]:
# Import/install Gradio
try:
    import gradio as gr
except:
    !pip -q install gradio
    import gradio as gr

print(f"Gradio version: {gr.__version__}")

### creating a function to map inputs and outputs..

In [ ]:
# put our model on cpu
effnetb2 = effnetb2.to('cpu')

# check the device
next(iter(effnetb2.parameters())).device

In [ ]:
# images of food -> ML model -> output
from typing import Tuple, Dict
def predict(img) -> Tuple[Dict, float]:

  # start a timer
  start_time = timer()

  # transform input image for use with effnetb2
  img = effnetb2_transforms(img).unsqueeze(0)

  # put model into eval mode and make pred
  effnetb2.eval()
  with torch.inference_mode():
    pred_probs = torch.softmax(effnetb2(img),dim=1)

  # create a pred label and pred prob dict
  pred_labels_and_probs = {class_names[i]: float(pred_probs[0][i]) for i in range(len(class_names))}

  # calculate the pred time
  end_time = timer()
  pred_time = round(end_time - start_time, 4)

  # return pred dict and pred time
  return pred_labels_and_probs, pred_time


In [ ]:
import random
from PIL import ImageChops
# get a list of all test images filepaths
test_data_paths = list(Path(test_dir).glob("*/*.jpg"))
test_data_paths[0]

#randomly slesct a test image path
random_image_path = random.sample(test_data_paths, k=1)[0]
random_image_path

#open the target image
image = Image.open(random_image_path)
print(f'Predictiing on image at path : {random_image_path}')

# predict on target image and print outputs
pred_dict, pred_time = predict(image)
print(pred_dict)
print(pred_time)

### Creating a list of example images

In [ ]:
# Create a list of example inputs to our Gradio demo
example_list = [[str(filepath)] for filepath in random.sample(test_data_paths, k=3)]
example_list

### Building a Gradio interface

In [ ]:
import gradio as gr

# create title, desc and article
title="Food Vision Mini 🍕🥩🍣"
description="An EffNetB2 feature extractor computer vision model to classify image of Pizza, Steak, Sushi..."
article="Soon"

# creating the gradio demo
demo = gr.Interface(fn=predict,
                    inputs=gr.Image(type='pil'),
                    outputs=[gr.Label(num_top_classes=3, label="Predictions"),
                             gr.Number(label="Prediction time (S)")],
                    examples=example_list,
                    title=title,
                    description=description,
                    article=article)

# Launch the demo
demo.launch(debug=False,
            share=True) # generates a publically shareable URL

## Permanently deploying Food Vision Mini using HugginFace

### Creating a `demos` folder to store app files

In [ ]:
import shutil
from pathlib import Path

# create foodvision min demo path
foodvision_mini_demo_path = Path('demos/foodvision_mini')

# remove files taht might exist and create a new directory
if foodvision_mini_demo_path.exists():
  shutil.rmtree(foodvision_mini_demo_path)
  foodvision_mini_demo_path.mkdir(parents=True,
                                  exist_ok=True)
else:
  foodvision_mini_demo_path.mkdir(parents=True,
                                  exist_ok=True)
  !ls demos/foodvision_mini/

### Creating a folder of example images to use with foodvision mini demo

In [ ]:
import shutil
from pathlib import Path

#create example directory
foodvision_mini_examples_path = foodvision_mini_demo_path / 'examples'
foodvision_mini_examples_path.mkdir(parents=True,
                                exist_ok=True)

# 2. Collect three random test dataset image paths
foodvision_mini_examples = [Path('data/pizza_steak_sushi_20_percent/test/sushi/592799.jpg'),
                            Path('data/pizza_steak_sushi_20_percent/test/steak/3622237.jpg'),
                            Path('data/pizza_steak_sushi_20_percent/test/pizza/2582289.jpg')]

# 3. Copy the three random images to the examples directory
for example in foodvision_mini_examples:
    destination = foodvision_mini_examples_path / example.name
    print(f"[INFO] Copying {example} to {destination}")
    shutil.copy2(src=example, dst=destination)

In [ ]:
import os

# Get example filepaths in a list of lists
example_list = [["examples/" + example] for example in os.listdir(foodvision_mini_examples_path)]
example_list

### Moving our trained EffNetB2 model to our foodvision mini demo directory..

In [ ]:
import shutil

# create source path for out target model
effnetb2_foodvision_mini_model_path = 'models/pretrained_effnetb2_feature_extractor_20%.pth'

# create a destination path for our target model
effnetb2_foodvision_mini_model_destination = foodvision_mini_demo_path / effnetb2_foodvision_mini_model_path.split("/")[1]

# trying to move the model file
try:
  print(f'[INFO] Moving {effnetb2_foodvision_mini_model_path} to {effnetb2_foodvision_mini_model_destination}')

  #move the model
  shutil.move(src=effnetb2_foodvision_mini_model_path,
              dst=effnetb2_foodvision_mini_model_destination)
  print(f'Model move completed successfully')
except:
  print(f'[INFO] No model found at : {effnetb2_foodvision_mini_model_path}, perhaps it already ahs been moved')
  print(f'Model exists at : {effnetb2_foodvision_mini_model_destination} : {effnetb2_foodvision_mini_model_destination.exists()}')

### Turning of EffNetB2 model into a python script

In [ ]:
%%writefile demos/foodvision_mini/model.py
import torch
import torchvision

from torch import nn


def create_effnetb2_model(num_classes:int=3,
                          seed:int=42):
    # Create EffNetB2 pretrained weights, transforms and model
    weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
    transforms = weights.transforms()
    model = torchvision.models.efficientnet_b2(weights=weights)

    # Freeze all layers in base model
    for param in model.parameters():
        param.requires_grad = False

    # Change classifier head with random seed for reproducibility
    torch.manual_seed(seed)
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.3, inplace=True),
        nn.Linear(in_features=1408, out_features=num_classes),
    )

    return model, transforms

In [ ]:
from demos.foodvision_mini import model

effnetb2_model, effnetb2_transforms_import = model.create_effnetb2_model()
effnetb2_model

### Turning out foodvision mini gradio app into python script

In [ ]:
%%writefile demos/foodvision_mini/app.py

import gradio as gr
import os
import torch
from model import create_effnetb2_model
from timeit import default_timer as timer
from typing import Tuple, Dict

# setup class names
class_names = ['pizza', 'steak', 'sushi']

# model and transforms preparation
effnetb2, effnetb2_transforms = create_effnetb2_model(
    num_classes=3)

# laod save weights
effnetb2.load_state_dict(
    torch.load(
        f="09_pretrained_effnetb2_feature_extractor_pizza_steak_sushi_20_percent.pth",
        map_location=torch.device("cpu"),  # load to CPU
    )
)

# predict function
def predict(img) -> Tuple[Dict, float]:

  # start a timer
  start_time = timer()

  # transform input image for use with effnetb2
  img = effnetb2_transforms(img).unsqueeze(0)

  # put model into eval mode and make pred
  effnetb2.eval()
  with torch.inference_mode():
    pred_probs = torch.softmax(effnetb2(img),dim=1)

  # create a pred label and pred prob dict
  pred_labels_and_probs = {class_names[i]: float(pred_probs[0][i]) for i in range(len(class_names))}

  # calculate the pred time
  end_time = timer()
  pred_time = round(end_time - start_time, 4)

  # return pred dict and pred time
  return pred_labels_and_probs, pred_time

## Gradio APP

# create title, desc and article
title="Food Vision Mini 🍕🥩🍣"
description="An EffNetB2 feature extractor computer vision model to classify image of Pizza, Steak, Sushi..."
article="Soon"

#create example list
example_list = [["examples/" + example] for example in os.listdir('examples')]

# creating the gradio demo
demo = gr.Interface(fn=predict,
                    inputs=gr.Image(type='pil'),
                    outputs=[gr.Label(num_top_classes=3, label="Predictions"),
                             gr.Number(label="Prediction time (S)")],
                    examples=example_list,
                    title=title,
                    description=description,
                    article=article)

# Launch the demo
demo.launch(debug=False,
            share=True) # generates a publically shareable URL

### Creating `requirements.txt`

In [ ]:
%%writefile demos/foodvision_mini/requirements.txt
torch==2.5.1+cu121
torchvision==0.20.1+cu121
gradio==5.13.1

## Deploying our model on Hugging face Spaces

In [ ]:
# Downloading our foodvision mini app files
!ls demos/foodvision_mini/

In [ ]:
# change into foodvision min directory and zip it from inside
!cd demos/foodvision_mini && zip -r ../foodvision_mini.zip * -x "*.pyc" "*.ipynb" "*__pycache__*" "*ipynb_checkpoints*"

In [ ]:
## download it locally
#from google.colab import files
#try:
#  files.download('demos/foodvision_mini.zip')
#except:
#  print('Not running on Google Colab')

In [ ]:
# IPython is a library to help make Python interactive
from IPython.display import IFrame

# Embed FoodVision Mini Gradio demo
IFrame(src="https://huggingface.co/spaces/Rageya/foodvision/+", width=900, height=750)

## FoodVision Big model on Food101 Dataset..

### Creaiing a model

In [ ]:
effnetb2_101_model, effnetb2_transforms = create_effnetb2_model(num_classes=101)

In [ ]:
from torchinfo import summary

 # Get a summary of EffNetB2 feature extractor for Food101 with 101 output classes (uncomment for full output)
summary(effnetb2_101_model,
  input_size=(1, 3, 224, 224),
  col_names=["input_size", "output_size", "num_params", "trainable"],
  col_width=20,
  row_settings=["var_names"])

In [ ]:
food101_train_transforms = torchvision.transforms.Compose([
    torchvision.transforms.TrivialAugmentWide(),
    effnetb2_transforms
])

In [ ]:
### Getting data

In [ ]:
from torchvision import datasets

#setup path directory
from pathlib import Path
data_dir = Path("data")

#get the training data
train_data = datasets.Food101(root=data_dir,
                              split="train",
                              transform=food101_train_transforms,
                              download=True)

#getting the testing data
test_data = datasets.Food101(root=data_dir,
                             split="test",
                             transform=effnetb2_transforms,
                             download=True)

In [ ]:
# getting the classes
food101_class_names = train_data.classes

food101_class_names[:10]

### Creating dataloaders

In [ ]:
from keras.callbacks import EarlyStopping

In [ ]:
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=3,          # Number of epochs with no improvement before stopping
    min_delta=0.01,      # Minimum change in the monitored quantity to qualify as an improvement
    restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored quantity
)

In [ ]:
import os
import torch

BATCH_SIZE = 32
NUM_WORKERS = 2 if os.cpu_count() <= 4 else 4

# Create Food101 training DataLoader
train_dataloader_food101 = torch.utils.data.DataLoader(train_data,
                                                                  batch_size=BATCH_SIZE,
                                                                  shuffle=True,
                                                                  num_workers=NUM_WORKERS)
# Create Food101 testing DataLoader
test_dataloader_food101 = torch.utils.data.DataLoader(test_data,
                                                                 batch_size=BATCH_SIZE,
                                                                 shuffle=False,
                                                                 num_workers=NUM_WORKERS)

### Training the model..

In [ ]:
from going_modular.going_modular import engine

#optimizer
optimizer=torch.optim.Adam(params=effnetb2_101_model.parameters(),
                           lr=1e-3)

#loss fn
loss_fn = nn.CrossEntropyLoss()

#train the model
effnetb2_101_model_results = engine.train(model=effnetb2_101_model,
                                          train_dataloader=train_dataloader_food101,
                                          test_dataloader=test_dataloader_food101,
                                          optimizer=optimizer,
                                          loss_fn=loss_fn,
                                          epochs=16,
                                          device=device)